Imorts

In [1]:
import numpy as np
import tkinter

# Spielelogik

In [2]:
# Cell states
NONE = 0
BLACK = 1
WHITE = 2

# An Exception that is raised every time an invalid move occurs
class InvalidMoveException(Exception):
    ''' Raised whenever an exception arises from an invalid move '''
    pass


# The Othello class that manages the game state
class OthelloGameState:
    '''
    Class that creates the Othello game and deals with all its game logic
    '''
    
    # Initialize the game through the __init__() function.
    def __init__(self):
        ''' Creates the initial game state. '''
        self.current_board = np.zeros((8, 8), dtype=np.int8)
        self.current_board[3, 3] = WHITE
        self.current_board[3, 4] = BLACK
        self.current_board[4, 3] = BLACK
        self.current_board[4, 4] = WHITE
        self.frontier = {(2,2),(2,3),(2,4),(2,5),
                         (3,2),(3,5),(4,2),(4,5),
                         (5,2),(5,3),(5,4),(5,5)}
        self.turn = BLACK


    # This is the meat of the game logic. I define making a move through the
    # move() function, and within it are broken down helper functions that
    # make the code more reusable and the move() function more readable.
    def move(self, row: int, col: int) -> None:
        ''' Attempts to make a move at given row/col position.
            Current player/turn is the one that makes the move.
            If the player cannot make a move it raises an exception.
            If the player can make a move, the player finally plays
            the valid move and switches turn. '''
        
        # Check to see if the move is in a valid empty space
        # within the board's boundary
        if (row, col) not in self.frontier:
            raise InvalidMoveException
        
        # Retrieve a list of possible directions in which a valid move can occur.
        # (looks up to all 8 possible directions surrounding the move/cell)
        # The list only contains the directions where the opposite cell's color is
        # adjacent / touching the current cell. So it's not definite that the entire
        # list will return directions in which the cells in line of direction can be flipped.
        possible_directions = self._adjacent_opposite_color_directions(row, col, self.turn)
        
        # After having the list of possible directions, we begin keeping track of when a possible
        # valid move in a direction has been completed. The variable "next_turn" is used at the end
        # of this function to determine if the player switches turn, which only occurs if the
        # move is valid.
        #
        # The for loop looks through all of the possible directions, and if a direction is capable
        # of making a valid move/flip, then proceed into flipping the cells in that line of direction
        # and assign "next_turn" to the opposite turn to switch turns at the very end
        next_turn = self.turn
        for direction in possible_directions:
            if self._is_valid_directional_move(row, col, direction[0], direction[1], self.turn):
                next_turn = self._opposite_turn(self.turn)
            self._convert_adjacent_cells_in_direction(row, col, direction[0], direction[1], self.turn)


        # Here we decide if we can finally place down the current move and whether or not
        # we can switch turns. We decide to switch turns if the current player has made a valid move
        # AND the opposite player must have the option to be able to move in at least one empty cell
        # space. If the opposite player can't move in at least one empty cell space after the current
        # player has gone, we do not switch turns and the current player goes again for the second
        # time in a row.
        #
        # Ultimately, if the move is not valid, then raise an InvalidMoveException()
        if next_turn != self.turn:
            self.current_board[row, col] = self.turn
            self._update_frontier(row, col)
            if self.can_move(next_turn):
                self.switch_turn()
        else:
            raise InvalidMoveException()

    def _update_frontier(self, row: int, col: int) -> None:
        '''Updates the frontier set of possible move locations after a move has been made'''
        for current_row in range(row-1, row+2):
            if not self._is_valid_row_number(current_row):
                continue
            for current_col in range(col-1, col+2):
                if not self._is_valid_col_number(current_col):
                    continue
                if self.current_board[current_row, current_col] == NONE:
                    self.frontier.add((current_row, current_col))
        self.frontier.remove((row, col))

    def _is_valid_directional_move(self, row: int, col: int, rowdelta: int, coldelta: int, turn: str) -> bool:
        ''' Given a move at specified row/col, checks in the given direction to see if
            a valid move can be made. Returns True if it can; False otherwise.
            Only supposed to be used in conjunction with _adjacent_opposite_color_directions()'''
        current_row = row + rowdelta
        current_col = col + coldelta

        last_cell_color = self._opposite_turn(turn)

        while True:
            # Immediately return false if the board reaches the end (b/c there's no blank
            # space for the cell to sandwich the other colored cell(s)
            if not self._is_valid_cell(current_row, current_col):
                break
            if self._cell_color(current_row, current_col) == NONE:
                break           
            if self._cell_color(current_row, current_col) == turn:
                last_cell_color = turn
                break

            current_row += rowdelta
            current_col += coldelta
            
        return last_cell_color == turn


    def _adjacent_opposite_color_directions(self, row: int, col: int, turn: str) -> [tuple]:
        ''' Looks up to a possible of 8 directions surrounding the given move. If any of the
            move's surrounding cells is the opposite color of the move itself, then record
            the direction it is in and store it in a list of tuples [(rowdelta, coldelta)].
            Return the list of the directions at the end. '''
        dir_list = []
        for rowdelta in range(-1, 2):
            if not self._is_valid_row_number(row+rowdelta):
                continue
            for coldelta in range(-1, 2):
                if not self._is_valid_col_number(col+coldelta):
                    continue
                if self.current_board[row + rowdelta, col + coldelta] == self._opposite_turn(turn):
                    dir_list.append((rowdelta, coldelta))
        return dir_list
           

    def _convert_adjacent_cells_in_direction(self, row: int, col: int,
                                             rowdelta: int, coldelta: int, turn: str) -> None:
        ''' If it can, converts all the adjacent/contiguous cells on a turn in
            a given direction until it finally reaches the specified cell's original color '''
        if self._is_valid_directional_move(row, col, rowdelta, coldelta, turn):
            current_row = row + rowdelta
            current_col = col + coldelta
            
            while self._cell_color(current_row, current_col) == self._opposite_turn(turn):
                self._flip_cell(current_row, current_col)
                current_row += rowdelta
                current_col += coldelta


    # Functions to be used to determine if the game is over and what do when it is:
    #
    # is_game_over()
    # can_move()
    # return_winner()
    #
    def is_game_over(self) -> bool:
        ''' Looks through every empty cell and determines if there are
            any valid moves left. If not, returns True; otherwise returns False '''
        return self.can_move(BLACK) == False and self.can_move(WHITE) == False


    def can_move(self, turn: str) -> bool:
        ''' Looks at the frontier of the board and checks to
            see if the specified player can move in any of the cells.
            Returns True if it can move; False otherwise. '''
        for (row, col) in self.frontier:
            for direction in self._adjacent_opposite_color_directions(row, col, turn):
                if self._is_valid_directional_move(row, col, direction[0], direction[1], turn):
                    return True
        return False

    def return_winner(self) -> str:
        ''' Returns the winner. ONLY to be called once the game is over.
            Returns None if the game is a TIE game.'''
        black_cells = self.get_total_cells(BLACK)
        white_cells = self.get_total_cells(WHITE)

        if black_cells == white_cells:
            return None

        if black_cells > white_cells:
            return BLACK
        else:
            return WHITE


    # Utility functions for implementing ai agents
    
    def get_possible_moves(self):
        possible_moves = []
        for (row, col) in self.frontier:
            for direction in self._adjacent_opposite_color_directions(row, col, self.turn):
                if self._is_valid_directional_move(row, col, direction[0], direction[1], self.turn):
                    possible_moves.append((row, col))
                    break
        return possible_moves

    # Basic functions that perform simple tasks, ranging from retrieving
    # specific game data and switching turns:
    #
    # switch_turn()
    # get_rows()
    # get_columns()
    # get_turn()
    # get_total_cells()
    #
    def switch_turn(self) -> None:
        ''' Switches the player's turn from the current one to
            the other. Only to be called if the current player
            cannot move at all. '''
        self.turn = self._opposite_turn(self.turn)

    def get_board(self) -> [[str]]:
        ''' Returns the current game's 2D board '''
        return self.current_board

    def get_rows(self) -> int:
        ''' Returns the number of rows the game currently has '''
        return 8

    def get_columns(self) -> int:
        ''' Returns the number of columns the game currently has '''
        return 8

    def get_turn(self) -> str:
        ''' Returns the current game's turn '''
        return self.turn

    def get_total_cells(self, turn: str) -> int:
        ''' Returns the total cell count of the specified colored player '''
        total = 0
        for row in range(8):
            for col in range(8):
                if self.current_board[row, col] == turn:
                    total += 1
        return total


    # The rest of the functions are private functions only to be used within this module
    def _flip_cell(self, row: int, col: int) -> None:
        ''' Flips the specified cell over to the other color '''
        self.current_board[row, col] = self._opposite_turn(self.current_board[row][col])


    def _cell_color(self, row: int, col: int) -> str:
        ''' Determines the color/player of the specified cell '''
        return self.current_board[row, col]
        

    def _opposite_turn(self, turn: str) -> str:
        ''' Returns the player of the opposite player '''
        return {BLACK: WHITE, WHITE: BLACK}[turn]

    def _is_valid_cell(self, row: int, col: int) -> bool:
        ''' Returns True if the given cell move position is invalid due to
            position (out of bounds) '''
        return self._is_valid_row_number(row) and self._is_valid_col_number(col)

    def _is_valid_row_number(self, row: int) -> bool:
        ''' Returns True if the given row number is valid; False otherwise '''
        return 0 <= row < 8

    def _is_valid_col_number(self, col: int) -> bool:
        ''' Returns True if the given col number is valid; False otherwise '''
        return 0 <= col < 8

    


# GUI

In [3]:
# GUI / tkinter object constants
BACKGROUND_COLOR = '#696969'
GAME_COLOR = '#006000'
FONT = ('Helvetica', 30)
DIALOG_FONT = ('Helvetica', 20)
PLAYERS = {BLACK: 'Black', WHITE: 'White'}
# GUI Constants
GAME_HEIGHT = 300
GAME_WIDTH = 300

In [4]:
class GameBoard:
    def __init__(self, game_state: OthelloGameState, game_width: float,
                 game_height: float, root_window) -> None:
        # Initialize the game board's settings here
        self._game_state = game_state
        self._board = tkinter.Canvas(master = root_window,
                                     width = game_width,
                                     height = game_height,
                                     background = GAME_COLOR)

    def redraw_board(self) -> None:
        ''' Redraws the board '''
        self._board.delete(tkinter.ALL)
        self._redraw_lines()
        self._redraw_cells()

    def _redraw_lines(self) -> None:
        ''' Redraws the board's lines '''
        row_multiplier = float(self._board.winfo_height()) / 8
        col_multiplier = float(self._board.winfo_width()) / 8
        
        # Draw the horizontal lines first
        for row in range(1, 8):
            self._board.create_line(0, row * row_multiplier, self.get_board_width(), row * row_multiplier)

        # Draw the column lines next
        for col in range(1, 8):
            self._board.create_line(col * col_multiplier, 0, col * col_multiplier, self.get_board_height())

    def _redraw_cells(self) -> None:
        ''' Redraws all the occupied cells in the board '''
        for row in range(8):
            for col in range(8):
                if self._game_state.get_board()[row][col] != NONE:
                    self._draw_cell(row, col)
                
    def _draw_cell(self, row: int, col: int) -> None:
        ''' Draws the specified cell '''
        self._board.create_oval(col * self.get_cell_width(),
                                row * self.get_cell_height(),
                                (col + 1) * self.get_cell_width(),
                                (row + 1) * self.get_cell_height(),
                                fill = PLAYERS[self._game_state.get_board()[row][col]])                               


    def update_game_state(self, game_state: OthelloGameState) -> None:
        ''' Updates our current _game_state to the specified one in the argument '''
        self._game_state = game_state

    def get_cell_width(self) -> float:
        ''' Returns a game cell's width '''
        return self.get_board_width() / 8

    def get_cell_height(self) -> float:
        ''' Returns a game cell's height '''
        return self.get_board_height() / 8

    def get_board_width(self) -> float:
        ''' Returns the board canvas's width '''
        return float(self._board.winfo_width())

    def get_board_height(self) -> float:
        ''' Returns the board canvas's height '''
        return float(self._board.winfo_height())

    def get_board(self) -> tkinter.Canvas:
        ''' Returns the game board '''
        return self._board


class Score:
    def __init__(self, color: str, game_state: OthelloGameState, root_window) -> None:
        ''' Initializes the score label '''
        self._player = color
        self._score = game_state.get_total_cells(self._player)
        self._score_label = tkinter.Label(master = root_window,
                                          text = self._score_text(),
                                          background = BACKGROUND_COLOR,
                                          fg = PLAYERS[color],
                                          font = FONT)

    def update_score(self, game_state: OthelloGameState) -> None:
        ''' Updates the score with the specified game state '''
        self._score = game_state.get_total_cells(self._player)
        self._change_score_text()

    def get_score_label(self) -> tkinter.Label:
        ''' Returns the score label '''
        return self._score_label

    def get_score(self) -> int:
        ''' Returns the score '''
        return self._score

    def _change_score_text(self) -> None:
        ''' Changes the score label's text '''
        self._score_label['text'] =  self._score_text()

    def _score_text(self) -> str:
        ''' Returns the score in text string format '''
        return PLAYERS[self._player] + ' - ' + str(self._score)



class Turn:
    def __init__(self, game_state: OthelloGameState, root_window) -> None:
        ''' Initializes the player's turn Label '''
        self._player = game_state.get_turn()
        self._turn_label = tkinter.Label(master = root_window,
                                          text = self._turn_text(),
                                          background = BACKGROUND_COLOR,
                                          fg = PLAYERS[self._player],
                                          font = FONT)

    def display_winner(self, winner: str) -> None:
        ''' Only called when the game is over. Displays the game winner '''
        if winner == None:
            victory_text = 'Tie game. Nobody wins!'
            text_color = 'BLACK'
        else:
            victory_text = PLAYERS[winner] + ' player wins!'
            text_color = PLAYERS[winner]
        self._turn_label['text'] = victory_text
        self._turn_label['fg'] = text_color

    def switch_turn(self, game_state: OthelloGameState) -> None:
        ''' Switch's the turn between the players '''
        self._player = game_state.get_turn()
        self.change_turn_text()

    def change_turn_text(self) -> None:
        ''' Changes the turn label's text '''
        self._turn_label['text'] = self._turn_text()
        self._turn_label['fg'] = PLAYERS[self._player]

    def get_turn_label(self) -> None:
        ''' Returns the tkinter turn label '''
        return self._turn_label

    def update_turn(self, turn: str) -> None:
        ''' Updates the turn to whatever the current game state's turn is '''
        self._player = turn
        self.change_turn_text()

    def _turn_text(self) -> None:
        ''' Returns the turn in text/string form '''
        return PLAYERS[self._player] + " player's turn"

    def _opposite_turn(self) -> None:
        ''' Returns the opposite turn of current turn '''
        return {BLACK: WHITE, WHITE: BLACK}[self._player]

    def show(self) -> None:
        self._dialog_window.grab_set()
        self._dialog_window.wait_window()
        

In [5]:
class OthelloGUI:
    def __init__(self):
        # Create my othello gamestate here (drawn from the original othello game code)
        self._game_state = OthelloGameState()

        
        # Initialize all my widgets and window here
        self._root_window = tkinter.Tk()
        self._root_window.configure(background = BACKGROUND_COLOR)
        self._board = GameBoard(self._game_state, GAME_WIDTH, GAME_HEIGHT, self._root_window)
        self._black_score = Score(BLACK, self._game_state, self._root_window)
        self._white_score = Score(WHITE, self._game_state, self._root_window)
        self._player_turn = Turn(self._game_state, self._root_window)


        # Bind my game board with these two events.
        self._board.get_board().bind('<Configure>', self._on_board_resized)
        self._board.get_board().bind('<Button-1>', self._on_board_clicked)        


        # Create our menu that can be accessed at the top of the GUI
        self._menu_bar = tkinter.Menu(self._root_window)
        self._game_menu = tkinter.Menu(self._menu_bar, tearoff = 0)
        self._game_menu.add_command(label = 'New Game', command = self._new_game)
        self._game_menu.add_separator()
        self._game_menu.add_command(label = 'Exit', command = self._root_window.destroy)
        self._menu_bar.add_cascade(label = 'Game', menu = self._game_menu)
        

        # Layout all the widgets here using grid layout
        self._root_window.config(menu = self._menu_bar)
        self._black_score.get_score_label().grid(row = 0, column = 0,
                               sticky = tkinter.S)
        self._white_score.get_score_label().grid(row = 0, column = 1,
                               sticky = tkinter.S)
        self._board.get_board().grid(row = 1, column = 0, columnspan = 2,
                                     padx = 50, pady = 10,
                                     sticky = tkinter.N + tkinter.E + tkinter.S + tkinter.W)
        self._player_turn.get_turn_label().grid(row = 2, column = 0, columnspan = 2,
                               padx = 10, pady = 10)


        # Configure the root window's row/column weight (from the grid layout)
        self._root_window.rowconfigure(0, weight = 1)
        self._root_window.rowconfigure(1, weight = 1)
        self._root_window.rowconfigure(2, weight = 1)
        self._root_window.columnconfigure(0, weight = 1)
        self._root_window.columnconfigure(1, weight = 1)


    def start(self) -> None:
        ''' Runs the mainloop of the root window '''
        self._root_window.mainloop()

    def _new_game(self) -> None:
        ''' Creates a new game with current _game_state settings '''
        self._game_state = OthelloGameState()
        self._board.redraw_board()
        self._black_score.update_score(self._game_state)
        self._white_score.update_score(self._game_state)
        self._player_turn.update_turn(self._game_state.get_turn())

    def _on_board_clicked(self, event: tkinter.Event) -> None:
        ''' Attempt to play a move on the board if it's valid '''
        move = self._convert_point_coord_to_move(event.x, event.y)
        row = move[0]
        col = move[1]
        try:
            self._game_state.move(row, col)
            self._board.update_game_state(self._game_state)
            self._board.redraw_board()
            self._black_score.update_score(self._game_state)
            self._white_score.update_score(self._game_state)
            
            if self._game_state.is_game_over():
                self._player_turn.display_winner(self._game_state.return_winner())
            else:
                self._player_turn.switch_turn(self._game_state)
        except InvalidMoveException:
            pass
        except:
            raise

    def _convert_point_coord_to_move(self, pointx: int, pointy: int) -> None:
        ''' Converts canvas point to a move that can be inputted in the othello game '''
        row = int(pointy // self._board.get_cell_height())
        if row == 8:
            row -= 1
        col = int(pointx // self._board.get_cell_width())
        if col == 8:
            col -= 1
        return (row, col)
        
    def _on_board_resized(self, event: tkinter.Event) -> None:
        ''' Called whenever the canvas is resized '''
        self._board.redraw_board()
